In [1]:
import pandas as pd

# Define the paths
csv_path = 'input/user_ratings.csv'
output_csv = 'output/user_rankings.csv'

# Read the CSV file
df = pd.read_csv(csv_path, delimiter=';')

# Calcular la puntuación total para cada historia por usuario
df['total_score'] = df.iloc[:, 2:-1].sum(axis=1)

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking: si hay empates se asigna la misma posición usando rank tipo dense
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total (para conservar la magnitud)
    group['ranking_value'] = group['total_score']
    # Calcular el ranking value normalizado por usuario (min-max)
    min_score = group['ranking_value'].min()
    max_score = group['ranking_value'].max()
    if max_score - min_score != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_score) / (max_score - min_score)
    else:
        group['ranking_value_norm'] = 0.0  # o el valor que prefieras para el caso de empate total
    return group

# Aplicar la función de ranking para cada usuario
df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)

# Seleccionar las columnas deseadas y renombrar 'user' a 'user_id'
df_output = df_ranked[['user', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']].rename(columns={'user': 'user_id'})

# Guardar el resultado en un CSV
df_output.to_csv(output_csv, index=False)

print(f"CSV file created: {output_csv}")

CSV file created: output/user_rankings.csv


/tmp/ipykernel_586489/2536200109.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)


In [2]:
import pandas as pd

# Definir las rutas
csv_path = 'input/user_ratings.csv'
output_csv = 'output/user_rankings.csv'
mapping_csv = 'input/github_original/anonymization/mapping-grouped.csv'

# Cargar el mapping y procesarlo
mapping_df = pd.read_csv(mapping_csv)
mapping_df['Original'] = mapping_df['Original'].str.replace('.txt', '')
mapping_df['Anonymized'] = mapping_df['Anonymized'].str.replace('.txt', '')
mapping_dict = dict(zip(mapping_df['Anonymized'], mapping_df['Original']))

def update_story_idx(story_id):
    original_id = mapping_dict.get(story_id, '')
    if original_id:
        return f"{story_id}_{original_id}"
    else:
        return story_id

# Leer el CSV de ratings
df = pd.read_csv(csv_path, delimiter=';')

# Calcular la puntuación total para cada historia por usuario
df['total_score'] = df.iloc[:, 2:-1].sum(axis=1)

def assign_ranking(group):
    group = group.copy()
    # Ordenar las historias por puntuación de forma descendente
    group = group.sort_values('total_score', ascending=False)
    # Asignar ranking: si hay empates se asigna la misma posición usando rank tipo dense
    group['ranking_pos'] = group['total_score'].rank(method='dense', ascending=False).astype(int)
    # El valor del ranking es la puntuación total (para conservar la magnitud)
    group['ranking_value'] = group['total_score']
    # Calcular el ranking value normalizado por usuario (min-max)
    min_score = group['ranking_value'].min()
    max_score = group['ranking_value'].max()
    if max_score - min_score != 0:
        group['ranking_value_norm'] = (group['ranking_value'] - min_score) / (max_score - min_score)
    else:
        group['ranking_value_norm'] = 0.0
    return group

# Aplicar la función de ranking para cada usuario
df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)

# Seleccionar las columnas deseadas y renombrar 'user' a 'user_id'
df_output = df_ranked[['user', 'story_id', 'ranking_pos', 'ranking_value', 'ranking_value_norm']] \
    .rename(columns={'user': 'user_id'})

# Actualizar la story_id usando el mapping
df_output['story_id'] = df_output['story_id'].apply(update_story_idx)

# Guardar el resultado en un CSV
df_output.to_csv(output_csv, index=False)

print(f"CSV file created: {output_csv}")

CSV file created: output/user_rankings.csv


/tmp/ipykernel_586489/782194196.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ranked = df.groupby('user').apply(assign_ranking).reset_index(drop=True)
